<font face="Times New Roman">
<div dir=ltr align=center>
<font color=3C99D size=6>
    Essential and Coupled Reactions

In [ ]:
using COBREXA
import HiGHS

model = load_model(StandardModel, "./Models/e_coli_core.json")
blocked_reaction = ["EX_fru_e", "EX_fum_e", "EX_gln__L_e", "EX_mal__L_e", "FRUpts2", "FUMt2_2", "GLNabc", "MALt2_2"]

# removing blocked reactions from model
remove_reactions!(model, blocked_reaction)

### Coupling Reactions

In [ ]:
rxns = ["ACONTb", "GAPD", "CS", "PGK", "ENO", "GLCpts", "PGM", "EX_glc__D_e", "BIOMASS_Ecoli_core_w_GAM", "ACONTa"]
rxns_index = []
for r in rxns
    index = findfirst(isequal(r), reactions(model))
    push!(rxns_index, index)
end

In [ ]:
rxns_rev = []
for r in rxns_index
    if lower_bounds(model)[r] < 0
        push!(rxns_rev, r)
    end
end

### Making Homogeneous

In [ ]:
M = Int(1e6)
for r in reactions(model)
    if model.reactions[r].lb < 0
        model.reactions[r].lb = -M
        model.reactions[r].ub = M
    else
        model.reactions[r].lb = 0
        model.reactions[r].ub = M
    end
end

#### Wildtype Biomass

In [ ]:
solved = flux_dict(model, flux_balance_analysis(model, HiGHS.Optimizer; modifications=[silence]))
wt_biomass = solved["BIOMASS_Ecoli_core_w_GAM"]
const f = 0.01

# Single Reaction Essentials

In [ ]:
essentials = []
for r in reactions(model)    
    single_reaction_delete_model = change_bound(model, r, lower=0, upper=0)
    modif_solved = flux_dict(single_reaction_delete_model, flux_balance_analysis(
        single_reaction_delete_model,
        HiGHS.Optimizer;
        modifications=[silence]))
    
    biomass = modif_solved["BIOMASS_Ecoli_core_w_GAM"]
    if biomass <= f * wt_biomass
        push!(essentials, r)
    end
end

# Double Reaction Essentials

In [ ]:
non_essentials_r = setdiff(reactions(model), essentials)
n = length(non_essentials_r)
n_essentials = []
for i in 1:n-1
    for j in i+1:n
        double_reaction_delete_model = change_bounds(model,
            [non_essentials_r[i], non_essentials_r[j]],
            lower=[0, 0], upper=[0, 0])
        
        modif_solved = flux_dict(double_reaction_delete_model,
            flux_balance_analysis(double_reaction_delete_model,
                HiGHS.Optimizer;
                modifications=[silence]))
                
        biomass = modif_solved["BIOMASS_Ecoli_core_w_GAM"]
        if biomass <= f * wt_biomass
            push!(n_essentials, (non_essentials_r[i], non_essentials_r[j]))
        end
    end
end

# FCA

In [ ]:
using JuMP
import HiGHS

n = length(rxns)
S = stoichiometry(model)

couple_m = zeros(10, 10)

# 0 stands for Uncoupled
# 1 stands for Directionally coupled
# 2 stands for Partially coupled
# 3 stands for Fully coupled

### finding directionally and partially coupled reactoins
for i in 1:n
    for j in 1:n
        if i == j
            continue
        end
                
        optModel = Model(HiGHS.Optimizer)
        set_silent(optModel)
        @variable(optModel, upper_bounds(model)[k] >= v[k=1:n_reactions(model)] >= lower_bounds(model)[k])
        @constraint(optModel, S * v .== 0)
        @constraint(optModel, v[rxns_index[j]] == 0)
        @objective(optModel, Max, v[rxns_index[i]])
        optimize!(optModel)
        if isapprox(objective_value(optModel), 0; atol=1e-8)
            if rxns_index[i] in rxns_rev
                @objective(optModel, Min, v[rxns_index[i]])
                optimize!(optModel)
                if isapprox(objective_value(optModel), 0; atol=1e-8)
                    if couple_m[j, i] == 1
                        couple_m[i, j] = 2
                        couple_m[j, i] = 2
                    else
                        couple_m[i, j] = 1
                    end
                end
            else
                if couple_m[j, i] == 1
                    couple_m[i, j] = 2
                    couple_m[j, i] = 2
                else
                    couple_m[i, j] = 1
                end
            end
        end
    end
end

In [ ]:
### finding fully coupled in partially coupled reactions
for i in 1:9
    for j in i+1:10
        if couple_m[i, j] == 2
            optModel = Model(HiGHS.Optimizer)
            set_silent(optModel)
            @variable(optModel, upper_bounds(model)[k] >= v[k=1:n_reactions(model)] >= lower_bounds(model)[k])
            @constraint(optModel, S * v .== 0)
            @objective(optModel, Max, v[rxns_index[i]])
            optimize!(optModel)
            
            if isapprox(objective_value(optModel), 0; atol=1e-8)
                optModel1 = Model(HiGHS.Optimizer)
                set_silent(optModel1)
                @variable(optModel1, upper_bounds(model)[k] >= v[k=1:n_reactions(model)] >= lower_bounds(model)[k])
                @constraint(optModel1, S * v .== 0)
                @objective(optModel1, Min, v[rxns_index[i]])
                optimize!(optModel1)
                
                c1 = objective_value(optModel1) / value.(v[rxns_index[j]])
                @constraint(optModel1, v[rxns_index[j]] == value.(v[rxns_index[j]])*0.3)
                optimize!(optModel1)
                c2 = objective_value(optModel1) / value.(v[rxns_index[j]])
                if isapprox((c2 - c1), 0; atol=1e-8)
                    couple_m[i, j] = 3
                    couple_m[j, i] = 3
                end
            else
                c1 = objective_value(optModel) / value.(v[rxns_index[j]])
                @constraint(optModel, v[rxns_index[j]] == value.(v[rxns_index[j]])*0.3)
                optimize!(optModel)
                c2 = objective_value(optModel) / value.(v[rxns_index[j]])
                if isapprox((c2 - c1), 0; atol=1e-8)
                    couple_m[i, j] = 3
                    couple_m[j, i] = 3
                end
            end
        end
    end
end

In [ ]:
couple_matrix = Array{String}(undef, 10, 10)
for (i, j) in enumerate(couple_m)
    if i in [1, 12, 23, 34, 45, 56, 67, 78, 89, 100]
        couple_matrix[i] = "Nan"
    else
        if j == 0
            couple_matrix[i] = "U"
        elseif j == 1
            couple_matrix[i] = "D"
        elseif j == 2
            couple_matrix[i] = "P"
        else
            couple_matrix[i] = "F"
        end
    end
end